In [1]:
import pandas as pd

In [6]:
reviews = pd.read_csv("./Reviews.csv")
#print(type(reviews_df))
reviews_df = pd.DataFrame(reviews)

In [7]:
listings = pd.read_csv("./Listings.csv")
low_memory=False
listings_df = pd.DataFrame(listings)

In [8]:
reviews_df.head()

,listing_id,review_id,date,reviewer_id
0,11798,330265172,2018-09-30,11863072
1,15383,330103585,2018-09-30,39147453
2,16455,329985788,2018-09-30,1125378
3,17919,330016899,2018-09-30,172717984
4,26827,329995638,2018-09-30,17542859


In [13]:
listings_df.head()

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÂ² Paris (Sacre CÅ“ur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


In [15]:
#extract the data for the listing_info table from listings_df
listings_info = listings_df[["listing_id", "host_id", "name", "host_since", "host_location", "minimum_nights",
                           "maximum_nights", "instant_bookable"]]
listings_info.head()

,listing_id,host_id,name,host_since,host_location,minimum_nights,maximum_nights,instant_bookable
0,281420,1466919,"Beautiful Flat in le Village Montmartre, Paris",2011-12-03,"Paris, Ile-de-France, France",2,1125,f
1,3705183,10328771,39 mÂ² Paris (Sacre CÅ“ur),2013-11-29,"Paris, Ile-de-France, France",2,1125,f
2,4082273,19252768,"Lovely apartment with Terrace, 60m2",2014-07-31,"Paris, Ile-de-France, France",2,1125,f
3,4797344,10668311,Cosy studio (close to Eiffel tower),2013-12-17,"Paris, Ile-de-France, France",2,1125,f
4,4823489,24837558,Close to Eiffel Tower - Beautiful flat : 2 rooms,2014-12-14,"Paris, Ile-de-France, France",2,1125,f


In [17]:
#extract the data for the listing_review_stats tables from listings_df, is still missing total reviews
listing_review_stats = listings_df[["listing_id", 'review_scores_rating', 'review_scores_accuracy', 'review_scores_checkin',
                                   'review_scores_communication','review_scores_location', 'review_scores_value']]
listing_review_stats.head()

,listing_id,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,281420,100.0,10.0,10.0,10.0,10.0,10.0
1,3705183,100.0,10.0,10.0,10.0,10.0,10.0
2,4082273,100.0,10.0,10.0,10.0,10.0,10.0
3,4797344,100.0,10.0,10.0,10.0,10.0,10.0
4,4823489,100.0,10.0,10.0,10.0,10.0,10.0


In [35]:
#get the total review count for each listing_id with a review and organize data
#there are listing_ids with no reviews, meaning the null values will need to be changed to 0 after merge
#with listing_review_stats
listing_total_reviews = reviews_df['listing_id'].value_counts()
listing_total_reviews = pd.DataFrame(listing_total_reviews)
listing_total_reviews = listing_total_reviews.reset_index()
listing_total_reviews = listing_total_reviews.rename(columns={'listing_id':'total_reviews'})
listing_total_reviews = listing_total_reviews.rename(columns={'index':'listing_id'})
listing_total_reviews = pd.DataFrame(listing_total_reviews)
listing_total_reviews

,listing_id,total_reviews
0,17222007,891
1,8637229,828
2,1249964,796
3,32011332,762
4,2399029,754
...,...,...
193551,45914743,1
193552,41929967,1
193553,40561637,1
193554,36333146,1


In [ ]:
#merge listing_total_reviews with listing_review_stats